<a href="https://colab.research.google.com/github/dhairyachandra/Module-2-Deep-Learning-ICP/blob/master/ICP_10_Encrypted_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Encrypted Deep Learning
---
Traing a Deep Learning Model
---
Let's build and train a toy deep learning model

In [0]:
import torch as th
from torch import nn
from torch import optim
import torch.nn.functional as F


In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)


In [0]:
# a toy model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).mean()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        

In [0]:
model = Net()


In [31]:
train()

tensor(0.5680)
tensor(0.2991)
tensor(0.2259)
tensor(0.1828)
tensor(0.1492)
tensor(0.1216)
tensor(0.0987)
tensor(0.0798)
tensor(0.0644)
tensor(0.0517)
tensor(0.0415)
tensor(0.0332)
tensor(0.0265)
tensor(0.0212)
tensor(0.0171)
tensor(0.0137)
tensor(0.0111)
tensor(0.0090)
tensor(0.0073)
tensor(0.0060)


In [32]:
predictions = model(data)
predictions

tensor([[0.1122],
        [0.0381],
        [0.9578],
        [0.9392]], grad_fn=<AddmmBackward>)

Encrypted Deep Learning using PySyft
---
Now let's use the library PySyft, which we introduced in the first section of this course, to cary out encrypted learning :


In [33]:
!pip install syft

In [34]:
import syft as sy
import torch as th
from torch import nn, optim
hook = sy.TorchHook(th)

In [35]:
# create a couple of workers
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

<VirtualWorker id:secure_worker #objects:0>

In [0]:
# encrypt the model and share it among participants
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)

In [39]:
encrypted_prediction.get().float_precision()

tensor([[0.1120],
        [0.0380],
        [0.9550],
        [0.9360]])

Additive Secrete Sharing
---


Additive Secret Sharing is a protocol for Multi-Party Computation. It allows multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x.
Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [40]:
x = 5
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted).

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [41]:
bob_x_share *= 2 
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. 
Decryption happens by summing all the shares together MODULUS some constant:

In [42]:
x = 5

Q = 23740629843760239486723 # large prime number

bob_x_share = 23552870267 # <- a random number

alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [43]:
(bob_x_share + alice_x_share) % Q

5

 Fixed Precision Encoding
 ---
 Additive secrete sharing works with integers. Thus, to apply this protocol on deep learning, we first need to convert the gradeints to integers. To do so, we used a predifned funciton fixed_precision. In the following, we illustrate its internat implemention in a very simple appraoch:

In [0]:
BASE=10
PRECISION=4

def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION
    

In [45]:
encode(0.25)

2500

In [46]:
decode(2500)

0.25

---
Apply Addivite Secrete Sharing using PySyft on a real dataset (MNIST or CIFAR10)
---

In [89]:
import syft as sy
import torch as th
from torch import nn, optim

hook = sy.TorchHook(th)

In [90]:
print("foo has: " + str(foo1._objects))
print("boo has: " + str(boo1._objects))
print("stup has: " + str(stup1._objects))

foo has: {99744910525: tensor([[ 424738985362657175, 3400499388844169692,  993852697771133677,
          ..., 4019424619212998013,  625225470242552788,
         2859841612653891361],
        [3285324380256243462,   55877500610759758, 3226370687423381535,
          ..., 1094903217028333877, 1315814664156929910,
         2931022327922938341],
        [ 588073449478440062,  259960477589263161,  247142724361341213,
          ..., 3764770987337687220, 2973146281396309915,
         1975531308220434222],
        ...,
        [1828790947725996701, 3221503819810295755, 1186680022047057095,
          ..., 1251394968824032470, 3979083838520665006,
         2033481185462435484],
        [3186836617907264001,   26854853925688677, 4372454461003477036,
          ..., 4568243923429177525, 1089963784363729887,
         1118392683470204327],
        [4133955034166306909, 3478287833375024483, 2718354206898916257,
          ..., 1319703193231291223, 1387564548734176186,
         4079463647074751642]]), 34

In [0]:
import torch
import torchvision
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

In [0]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, )),
])

train_set = datasets.MNIST(
    "~/.pytorch/MNIST_data/", train=True, download=True, transform=transform)
test_set = datasets.MNIST(
    "~/.pytorch/MNIST_data/", train=False, download=True, transform=transform)

#federated_train_loader = sy.FederatedDataLoader(
    #train_set.federate((foo, boo)), batch_size=64, shuffle=True)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=64, shuffle=True)

In [0]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


model = Model()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [94]:
for epoch in range(0, 5):
    for batch_idx, (data, target) in enumerate(train_loader):
        # send the model to the client device where the data is present
        #model.send(data)
        # training the model
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output,dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # get back the improved model
        #model.get()
        if batch_idx % 100 == 0:
            # get back the loss
            #loss = loss.get()
            print('Epoch: {:2d} [{:5d}/{:5d} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                batch_idx * 64,
                len(train_loader) * 64,
                100. * batch_idx / len(train_loader),
                loss.item()))

Epoch:  1 [    0/60032 (  0%)]	Loss: 2.355145
Epoch:  1 [ 6400/60032 ( 11%)]	Loss: 1.173603
Epoch:  1 [12800/60032 ( 21%)]	Loss: 0.806585
Epoch:  1 [19200/60032 ( 32%)]	Loss: 0.556049
Epoch:  1 [25600/60032 ( 43%)]	Loss: 0.562913
Epoch:  1 [32000/60032 ( 53%)]	Loss: 0.498203
Epoch:  1 [38400/60032 ( 64%)]	Loss: 0.568541
Epoch:  1 [44800/60032 ( 75%)]	Loss: 0.580611
Epoch:  1 [51200/60032 ( 85%)]	Loss: 0.375942
Epoch:  1 [57600/60032 ( 96%)]	Loss: 0.470994
Epoch:  2 [    0/60032 (  0%)]	Loss: 0.329182
Epoch:  2 [ 6400/60032 ( 11%)]	Loss: 0.361442
Epoch:  2 [12800/60032 ( 21%)]	Loss: 0.574150
Epoch:  2 [19200/60032 ( 32%)]	Loss: 0.524261
Epoch:  2 [25600/60032 ( 43%)]	Loss: 0.384313
Epoch:  2 [32000/60032 ( 53%)]	Loss: 0.447860
Epoch:  2 [38400/60032 ( 64%)]	Loss: 0.424182
Epoch:  2 [44800/60032 ( 75%)]	Loss: 0.257668
Epoch:  2 [51200/60032 ( 85%)]	Loss: 0.426812
Epoch:  2 [57600/60032 ( 96%)]	Loss: 0.300694
Epoch:  3 [    0/60032 (  0%)]	Loss: 0.282384
Epoch:  3 [ 6400/60032 ( 11%)]	Los

In [0]:
#Protecting the model weights 

encrypted_model = model.fix_precision().share(foo1, boo1, crypto_provider=stup1)

In [96]:
print(encrypted_model)

Model(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [0]:
encrypted_data = data.fix_precision().share(foo1, boo1, crypto_provider=stup1)

In [98]:
print(encrypted_data)

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:64415597145 -> foo1:17694735804]
	-> [PointerTensor | me:90764434548 -> boo1:69273365976]
	*crypto provider: stup1*


In [0]:
encrypted_prediction = encrypted_model(encrypted_data)

In [78]:
print(encrypted_prediction)

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:88939300736 -> foo1:84766676386]
	-> [PointerTensor | me:42337196877 -> boo1:54062004850]
	*crypto provider: stup1*
